In [1]:
import re
import os
import chromadb

from openai import OpenAI
from chromadb.utils.embedding_functions import ollama_embedding_function

In [2]:
# Connect to the Ollama server using OpenAI-compatible client
ollama_client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"  
)

# Creating an embedding function
embedding_function = ollama_embedding_function.OllamaEmbeddingFunction(
    model_name='nomic-embed-text',
    url='http://localhost:11434'
)

# Creating a persistent client
chroma_client = chromadb.PersistentClient(path='chroma-RAG-db')
collection_name = 'First-RAG-app'
collection = chroma_client.get_or_create_collection(
    name=collection_name,
    embedding_function=embedding_function
)

In [3]:
def load_documents_from_directory(directory_path, allowed_extensions=None):
    """
    Load text content from all files in a directory.
    
    Parameters:
        directory_path (str): Path to the directory.
        allowed_extensions (set): Extensions to include (e.g. {'.txt', '.md'}). Defaults to common text types.

    Returns:
        list[str]: A list of document contents.
    """
    import pathlib
    
    if allowed_extensions is None:
        allowed_extensions = {'.txt'}
    
    documents = []
    for file in os.listdir(directory_path):
        filepath = os.path.join(directory_path, file)
        ext = pathlib.Path(filepath).suffix.lower()

        if ext in allowed_extensions and os.path.isfile(filepath):
            with open(filepath, 'r', encoding='utf-8') as f:
                documents.append({"id":file, "text":f.read()})
    print(f"Loaded {len(documents)} documents from {directory_path}")
    return documents

# To split into small chunks
def split_text(text, chunk_size=1000, chunk_overlap=50): 
    start = 0
    chunks = []
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - chunk_overlap
    return chunks

news_articles_dir = './news_articles'
documents = load_documents_from_directory(news_articles_dir)

Loaded 21 documents from ./news_articles


In [4]:
chunked_docs = []
for doc in documents:
    chunks = split_text(doc['text'])
    for i, chunk in enumerate(chunks):
        chunked_docs.append({
            'id': f"{doc['id']}_chunk_{i}",
            'text': chunk
        })

def get_embedding(text):
    return embedding_function([text])[0]

In [5]:
for doc in chunked_docs:
    print(f"Processing embedding for document: {doc['id']}")
    doc['embedding'] = get_embedding(doc['text'])

print(doc['embedding'])

Processing embedding for document: 05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt_chunk_0
Processing embedding for document: 05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt_chunk_1
Processing embedding for document: 05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt_chunk_2
Processing embedding for document: 05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt_chunk_3
Processing embedding for document: 05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt_chunk_4
Processing embedding for document: 05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt_chunk_5
Processing embedding for document: 05-03-ai-replace-tv-writers-strike.txt_chunk_0
Processing embedding for document: 05-03-ai-replace-tv-writers-strike.txt_chunk_1
Processing embedding for document: 05-03-ai-replace-tv-writers-strike.txt_chunk_2
Processing embedding for document: 05-03-ai-replace-tv-writers-strike.txt_chunk_3


In [6]:
# Upsert documents to chroma db
for doc in chunked_docs:
    print('Upserting document: ', doc['id'], 'to collection: ', collection_name)
    collection.upsert(
        ids=[doc['id']],
        documents=[doc['text']],
        embeddings=[doc['embedding']],
    )

Upserting document:  05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt_chunk_0 to collection:  First-RAG-app
Upserting document:  05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt_chunk_1 to collection:  First-RAG-app
Upserting document:  05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt_chunk_2 to collection:  First-RAG-app
Upserting document:  05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt_chunk_3 to collection:  First-RAG-app
Upserting document:  05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt_chunk_4 to collection:  First-RAG-app
Upserting document:  05-03-ai-powered-supply-chain-startup-pando-lands-30m-investment.txt_chunk_5 to collection:  First-RAG-app
Upserting document:  05-03-ai-replace-tv-writers-strike.txt_chunk_0 to collection:  First-RAG-app
Upserting document:  05-03-ai-replace-tv-writers-strike.txt_chunk_1 to collection:  First-RAG-app
Upserting document:  05-03-ai-replac

In [7]:
def query_documents(question, n_results=2):
    results = collection.query(
                            query_texts=question, 
                            n_results=n_results
                            )
    relevant_chunks = [doc for doc in results['documents'][0]]

    # Return relevant chunks
    return relevant_chunks

In [8]:
def get_LLMResponse(question, relevant_chunks):
    context = '\n\n'.join(relevant_chunks)
    prompt = f"You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Use three sentences maximum and keep the answer concise. \n\nContext:\n {context} \n\nQuestion:\n {question}"

    response = ollama_client.chat.completions.create(
        model='deepseek-r1:1.5b',
        messages=[
            {
                "role": "system", 
                "content": prompt
            },
            {
                "role": "user", 
                "content": question
            }
        ]
    )
    return response.choices[0].message.content

In [9]:
def extract_thought_and_answer(text):
    thought_match = re.search(r"<think>(.*?)</think>", text, re.DOTALL)
    thought = thought_match.group(1).strip() if thought_match else None
    answer = re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()

    return thought, answer

* deepseek-r1:1.5b took 34.8s to generate a response for "tell me about chatgpt"
* deepseek-r1:7b took 2m 22s to generate a response for "tell me about chatgpt"

In [10]:
question = "who found microsoft"
relevant_chunks = query_documents(question)
print(relevant_chunks)
RAG_response = get_LLMResponse(question, relevant_chunks)

['hrough the Azure OpenAI Service, Microsoft’s fully managed, corporate-focused offering. Customers, who must already be “Microsoft managed customers and partners,” can apply here for special access.\n\nMarch 1, 2023\n\nOpenAI makes another move toward monetization by launching a paid API for ChatGPT. Instacart, Snap (Snapchat’s parent company) and Quizlet are among its initial customers.\n\nFebruary 7, 2023\n\nAt a press event in Redmond, Washington, Microsoft announced its long-rumored integration of OpenAI’s GPT-4 model into Bing, providing a ChatGPT-like experience within the search engine. The announcement spurred a 10x increase in new downloads for Bing globally, indicating a sizable consumer demand for new AI experiences.\n\nOther companies beyond Microsoft joined in on the AI craze by implementing ChatGPT, including OkCupid, Kaito, Snapchat and Discord — putting the pressure on Big Tech’s AI initiatives, like Google.\n\nFebruary 1, 2023\n\nAfter ChatGPT took the internet by sto

In [11]:
thought, answer = extract_thought_and_answer(RAG_response)

print("THOUGHT :\n", thought)
print("\nANSWER :\n", answer)

THOUGHT :
 Okay, so I'm trying to figure out who first discovered Microsoft. Let me start by thinking about what I know. I remember that Microsoft was a company from the late 20th century. My brother told me once that Microsoft's logo, the red square on the top of the Windows logo, stands for "Microsoft, Your Cloud (you = you), your home," which suggests it comes from Latin roots.

I think the concept and name of Microsoft were formed way back. I've heard about various companies before where they took existing businesses and turned them into brands with their own names. For example, Google was started by a former chairman who took a bank in England and gave it the word "Google." Similarly, Facebook was founded by AmbleWeb Inc.

So there's probably some overlap here because Microsoft is built up from components like Azure, which you know, they're related to cloud computing companies now. I believe those components were started before Microsoft as well, so maybe their names or technologi